In [1]:
import pandas as pd
import numpy as np
import math
# from data.get_data import run_query, install_backlog_sql, maint_backlog_sql, live_fleet_sql, new_sales_sla_sql, locs_in_implementation_sql

import functions.main_functions as mf
import functions.report_functions as rf

input_data = 'binder TEMPLATE Tech Hiring.xlsx'
install_res_dict, maint_res_dict, wo_tech_mnthly_rr_less_ss = mf.run_model(input_data)

### Visuals

In [2]:
import altair as alt
# alt.renderers.enable("jupyter")

# intall_df.index.name = 'Service Territory'
cap_perc_ma3, _ = rf.get_cap_perc_ma3(install_res_dict, maint_res_dict, wo_tech_mnthly_rr_less_ss)

cap_perc_ma3_c = cap_perc_ma3.copy()
cap_perc_ma3_c['gtr1'] = np.where(cap_perc_ma3_c.iloc[:,-1] > 1, 'yes', 'no')
gtr_col_df = cap_perc_ma3_c[['gtr1']]

cap_perc = rf.get_cap_perc(install_res_dict, maint_res_dict, wo_tech_mnthly_rr_less_ss)

chart_df = cap_perc_ma3.reset_index()

long_install_df = pd.melt(chart_df, id_vars='Service Territory', value_vars=cap_perc_ma3.columns, value_name='cap_perc', var_name='date')
long_install_df.set_index('Service Territory', inplace=True)
long_install_df = long_install_df.merge(gtr_col_df, on='Service Territory')
long_install_df = long_install_df.reset_index()



In [3]:
source = long_install_df

chart = alt.Chart(source).encode(
    alt.Detail('Service Territory'),
    alt.Color('gtr1').scale(scheme="lightgreyred")
    ).properties(
    width=900,
    height=600
)
# .transform_filter(alt.datum.symbol)
line = chart.mark_line().encode(alt.X('date:O'), alt.Y('cap_perc:Q', axis=alt.Axis(orient='right', format='%')))

line

alt.Chart(...)

### Write to excel

In [4]:
def add_ttl_row(df):
    df.loc['ttl'] = df.sum()
    return df

In [5]:

writer = pd.ExcelWriter('output/tech_hiring_model_output.xlsx', engine='openpyxl') 

# cap_perc_ma3.to_excel(writer, sheet_name='cap_perc_ma3')
# cap_perc.to_excel(writer, sheet_name='cap_perc')

wo_tech_mnthly_rr_less_ss_long = install_res_dict['wo_tech_wkyl_rr'][install_res_dict['wo_tech_wkyl_rr'].index == 'WO / tech / week']
wo_tech_mnthly_rr_less_ss_long.to_excel(writer, sheet_name='monthly_run_rate')
install_res_dict['qtrly_tech_cap'] = add_ttl_row(install_res_dict['qtrly_tech_cap'])
install_res_dict['qtrly_tech_cap'].to_excel(writer, sheet_name='tech_count')
install_res_dict['initial_travel_tech_count'].to_excel(writer, sheet_name='initial_travel_tech_count')


install_res_dict['st_grounded_travel_tech_cnt_df'].to_excel(writer, sheet_name='st_grounded_travel_tech_cnt')


maint_res_dict['live_fleet_mom_df'] = add_ttl_row(maint_res_dict['live_fleet_mom_df'])
maint_res_dict['live_fleet_mom_df'].to_excel(writer, sheet_name='live_fleet_mom')

install_res_dict['install_dt_unconstrained_v2'] = add_ttl_row(install_res_dict['install_dt_unconstrained_v2'])
install_res_dict['install_dt_unconstrained_v2'].to_excel(writer, sheet_name='install_dt')

maint_res_dict['maint_dt_unconstrained_v2'] = add_ttl_row(maint_res_dict['maint_dt_unconstrained_v2'])
maint_res_dict['maint_dt_unconstrained_v2'].to_excel(writer, sheet_name='maint_dt')

install_res_dict['local_tech_supply'] = add_ttl_row(install_res_dict['local_tech_supply'])
install_res_dict['local_tech_supply'].to_excel(writer, sheet_name='local_tech_supply')

install_res_dict['travel_tech_supply'] = add_ttl_row(install_res_dict['travel_tech_supply'])
install_res_dict['travel_tech_supply'].to_excel(writer, sheet_name='travel_tech_supply')


maint_res_dict['local_tech_maint_supply'] = add_ttl_row(maint_res_dict['local_tech_maint_supply'])
maint_res_dict['local_tech_maint_supply'].to_excel(writer, sheet_name='local_tech_maint_supply')

maint_res_dict['travel_tech_maint_supply'] = add_ttl_row(maint_res_dict['travel_tech_maint_supply'])
maint_res_dict['travel_tech_maint_supply'].to_excel(writer, sheet_name='travel_tech_maint_supply')

install_res_dict['internal_tech_supply_df'] = add_ttl_row(install_res_dict['internal_tech_supply_df'])
install_res_dict['internal_tech_supply_df'].to_excel(writer, sheet_name='internal_tech_supply')

install_res_dict['external_tech_supply_df'] = add_ttl_row(install_res_dict['external_tech_supply_df'])
install_res_dict['external_tech_supply_df'].to_excel(writer, sheet_name='external_tech_supply')


maint_res_dict['internal_maint_tech_supply_df'] = add_ttl_row(maint_res_dict['internal_maint_tech_supply_df'])
maint_res_dict['internal_maint_tech_supply_df'].to_excel(writer, sheet_name='internal_maint_tech_supply')

maint_res_dict['external_maint_tech_supply_df'] = add_ttl_row(maint_res_dict['external_maint_tech_supply_df'])
maint_res_dict['external_maint_tech_supply_df'].to_excel(writer, sheet_name='external_maint_tech_supply')


# maint_res_dict['nsa_external_maint_tech_supply_df'].to_excel(writer, sheet_name='nsa_external_maint_tech_supply')
# maint_res_dict['dish_external_maint_tech_supply_df'].to_excel(writer, sheet_name='dish_external_maint_tech_supply')

install_res_dict['intall_df'] = add_ttl_row(install_res_dict['intall_df'])
install_res_dict['intall_df'].to_excel(writer, sheet_name='unmet_install_dt')

maint_res_dict['maint_df'] = add_ttl_row(maint_res_dict['maint_df'])
maint_res_dict['maint_df'].to_excel(writer, sheet_name='unmet_maint_dt')

install_res_dict['st_grounded_travel_tech_capacity_df'].to_excel(writer, sheet_name='st_grounded_travel_tech_capacity')
install_res_dict['st_ttl_local_tech_capacity_df'].to_excel(writer, sheet_name='st_ttl_local_tech_capacity')

# roll_install_delta_dt_v_int_ext.to_excel(writer, sheet_name='roll_install_delta_dt_v_int_ext')


writer.close()

In [6]:

# writer = pd.ExcelWriter('output/tech_hiring_model_output.xlsx', engine='openpyxl') 

# cap_perc_ma3.to_excel(writer, sheet_name='cap_perc_ma3')
# cap_perc.to_excel(writer, sheet_name='cap_perc')

# wo_tech_mnthly_rr_less_ss.to_excel(writer, sheet_name='monthly_run_rate')
# install_res_dict['qtrly_tech_cap'].to_excel(writer, sheet_name='tech_count')

# install_res_dict['install_dt_unconstrained_v2'].to_excel(writer, sheet_name='install_dt')
# maint_res_dict['maint_dt_unconstrained_v2'].to_excel(writer, sheet_name='maint_dt')

# install_res_dict['local_tech_supply'].to_excel(writer, sheet_name='local_tech_supply')
# install_res_dict['travel_tech_supply'].to_excel(writer, sheet_name='travel_tech_supply')

# maint_res_dict['local_tech_maint_supply'].to_excel(writer, sheet_name='local_tech_maint_supply')
# maint_res_dict['travel_tech_maint_supply'].to_excel(writer, sheet_name='travel_tech_maint_supply')

# install_res_dict['internal_tech_supply_df'].to_excel(writer, sheet_name='internal_tech_supply')
# install_res_dict['external_tech_supply_df'].to_excel(writer, sheet_name='external_tech_supply')

# maint_res_dict['internal_maint_tech_supply_df'].to_excel(writer, sheet_name='internal_maint_tech_supply')
# maint_res_dict['external_maint_tech_supply_df'].to_excel(writer, sheet_name='external_maint_tech_supply')

# maint_res_dict['nsa_external_maint_tech_supply_df'].to_excel(writer, sheet_name='nsa_external_maint_tech_supply')
# maint_res_dict['dish_external_maint_tech_supply_df'].to_excel(writer, sheet_name='dish_external_maint_tech_supply')

# install_res_dict['intall_df'].to_excel(writer, sheet_name='unmet_install_dt')
# maint_res_dict['maint_df'].to_excel(writer, sheet_name='unmet_maint_dt')

# maint_res_dict['live_fleet_mom_df'].to_excel(writer, sheet_name='live_fleet_mom')
# # roll_install_delta_dt_v_int_ext.to_excel(writer, sheet_name='roll_install_delta_dt_v_int_ext')


# writer.close()